# FastAPI: Building our application backend

FastAPI is a modern Python backend framework for building APIs quickly. It’s:

* Fast (based on Starlette + Pydantic).
* Easy to use (less boilerplate, type hints).
* Async-ready (works with async/await).

# CRUD Operations

## (R) GET Endpoint

Simple requests (Reading) operations
```python
@app.get("/items/{item_id}")
def read_item(item_id: int, q: str | None = None):
    return {"item_id": item_id, "q": q}
```

USe requestes.get() to run the call 
```python
BASE = "http://127.0.0.1:8000/"   # Your API URL
resp = requests.get(BASE + "items/1")
```

## (C) POST Endpoint
Add new data to the application

```python
from pydantic import BaseModel
...
class Item(BaseModel):
    name: str
    description: Optional[str] = None
    price: float
...
@app.post("/items/")
def create_item(item: Item):
```

Use requests.post to run the call
```python
    new_item_data = {
        "name": "Widget",
        "description": "A shiny, new widget.",
        "price": 19.99
    }
    response = requests.post(f"{BASE_URL}/items/", json=new_item_data)
    print(f"Status Code: {response.status_code}")
```

## (U) PUT Endpoint

```python

@app.put("/items/{item_id}")
def update_item(item_id: int, updated_item: Item):
```

Use requests.put to run the call

```python
updated_item_data = {
        "name": "Super Widget",
        "description": "An updated, super shiny widget!",
        "price": 24.99
    }
    response = requests.put(f"{BASE_URL}/items/{item_id}", json=updated_item_data)
```

## (D) DELETE Endpoint

```python 
@app.delete("/items/{item_id}")
def delete_item(item_id: int):
```

use requests.delete to run the call

```python
response = requests.delete(f"{BASE_URL}/items/{item_id}")
```


# Project structure


```
app/
 ├── main.py        # Entry point
 ├── database.py    # Database connection
 ├── models.py      # SQLAlchemy models
 ├── schemas.py     # Pydantic schemas
 ├── routes/       # Routers (modular API endpoints)
 │     ├── items.py
 │     └── users.py
 
```
In addtion, keep your local secrets in a `.env` file and create a `.venv` virtual environment

main.py
* Intializes the app
* Connects to underlying infrastructure (auth to DBs, LLMs, so on)
* In small apps, endpoints are defined directly in main.py. iI large apps, `routes` are created

routes/  
Contains modular endpoints. Each file is a route

database.py  
Instanciates a SQLAlchemy engine and session  
In this example, an in memory SQLite DB is used, but usually a remote one is used


schemas.py  
Pydantic dataclasses to define every entity (users, items, etc.) with a types

models.py  
Database tables definitions to run queries with SQLAlchemy


## Simplified FastAPI Project

```
app/
 ├── main.py        # Entry point
 ├── database.py    # Database connection
 ├── models.py      # SQLAlchemy models
 ├── schemas.py     # Pydantic schemas
```

## Other usefull folders:
conf.py
```python
from pydantic import BaseSettings
from functools import lru_cache

# The BaseSettings class automatically loads environment variables
# that match the field names. For example, 'DATABASE_URL' will be
# loaded from the 'DATABASE_URL' environment variable.
class Settings(BaseSettings):
    """
    Application settings class.

    This class uses Pydantic to manage settings and load them from
    environment variables.
    """
    # Database configuration
    DATABASE_URL: str = "sqlite:///./sql_app.db"

    # You can add a description for each field, which is great for documentation
    # For example, if you add a different field
    # API_V1_STR: str = "/api/v1"


# Using lru_cache to ensure that the settings object is only created once
# This is a common pattern to avoid multiple instantiations of the class
# and to improve performance.
@lru_cache()
def get_settings():
    """
    Get and cache the application settings.
    
    This function returns a single instance of the Settings object.
    """
    return Settings()

# Instantiate the settings object
settings = get_settings()
```


test_main.py
```python
from fastapi.testclient import TestClient

from .main import app

client = TestClient(app)


def test_read_main():
    response = client.get("/")
    assert response.status_code == 200
    assert response.json() == {"msg": "Hello World"}
```

# Data Validation

Pydantic dataclasses have built-in functionallities to validate input data.
In addition, data is also typed in endpoint functions. This is useful to avoid errors and to **roperly document** the API.


# Dependency injection

If we need a resource defined outside the function wrapped in the endpoint, we can use dependency injection. For example, if we need a database connection, we can use the following code:
```python
@router.get("/", response_model=List[schemas.User])
def get_users(db: Session = Depends(get_db)):
    return db.query(models.User).all()
```


**Dependency Injection (DI)** is a software design pattern where an object receives its required "dependencies" (other objects or services) from an external source rather than creating them itself
* Objects (clients, resources, etc.) are instanciated only once
* We can maintain the endpoint and change the resource (change environment, mock up)

# Run the app

```bash
.venv/Scripts/activate
uvicorn main:app --reload
```


In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import requests

BASE = "http://127.0.0.1:8000/users"

# Create a user
resp = requests.post(BASE + "/", json={"name": "Bob", "email": "bob@example.com"})
print("Create:", resp.json())

Create: {'name': 'Bob', 'email': 'bob@example.com', 'id': 1}


In [3]:
# Create a user, validate email
resp = requests.post(BASE + "/", json={"name": "Bob", "email": "@bobpot"})
print("Create:", resp.json())

Create: {'detail': [{'type': 'value_error', 'loc': ['body', 'email'], 'msg': 'value is not a valid email address: There must be something before the @-sign.', 'input': '@bobpot', 'ctx': {'reason': 'There must be something before the @-sign.'}}]}


In [4]:
# Get all users 
resp = requests.get(BASE + "/")
print("Get by ID:", resp.json())

Get by ID: [{'name': 'Bob', 'email': 'bob@example.com', 'id': 1}]


In [5]:
# Get user by ID (with error handling example)
resp = requests.get(BASE + "/1")
print("Get by ID:", resp.json())

Get by ID: {'name': 'Bob', 'email': 'bob@example.com', 'id': 1}


In [6]:
# Update user
resp = requests.put(BASE + "/1", json={"name": "Bobby", "email": "bobby@example.com"})
print("Update:", resp.json())


Update: {'name': 'Bobby', 'email': 'bobby@example.com', 'id': 1}


In [7]:
resp.status_code

200

In [8]:
resp.text

'{"name":"Bobby","email":"bobby@example.com","id":1}'

In [9]:
# Delete user
resp = requests.delete(BASE + "/1")
print("Delete status:", resp.status_code)

Delete status: 204


In [10]:
# Try to fetch deleted user (should 404)
resp = requests.get(BASE + "/1")
print("Fetch deleted user:", resp.status_code, resp.json())


Fetch deleted user: 404 {'detail': 'User with id 1 not found'}


# Uses cases in AI

* ML Scoring
* IA Gen apps


```python

try:
    model = joblib.load("dummy_model.pkl")
except FileNotFoundError:
    raise

# Define a Pydantic model for the input data.
# This ensures that the incoming request has the correct data types.
class PredictInput(BaseModel):
    feature_1: float
    feature_2: float

# The scoring endpoint. It expects a POST request with JSON data.
@app.post("/predict/")
def predict(data: PredictInput):
    """
    Takes input features, makes a prediction using the loaded model,
    and returns the predicted value.
    """
    # Convert the input data to a numpy array, as expected by scikit-learn models.
    features = np.array([[data.feature_1, data.feature_2]])

    # Make a prediction.
    prediction = model.predict(features)

    # Return the prediction. We convert it to a float to ensure it's JSON serializable.
    return {"prediction": float(prediction[0])}

```